<a href="https://colab.research.google.com/github/mr0112358/SDL-Course/blob/master/SDL_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1


## Part (a)

In [ ]:
import numpy as np

In [ ]:
def sigm(s): # Define sigmoid function
  return 1.0/(1 +np.e**-s)

def dsigm(s): # Derivative of sigmoid function
  return sigm(s) * (1-sigm(s))

In [ ]:
W1 = np.array([[1.5, 2.5, 1.0],[2.0, -1.5, -3.0]]) # [[w_h1b, w_h1x1, w_h1x2],[w_h2b, w_h2x1, w_h2x2]]
W2 = np.array([-1.0, 1.0, 0.5])                    # [w_yb, w_yh1, w_yh2]

In [ ]:
# Input Layer
x1 = 0; x2 = 1
X = np.array([1.0, x1, x2])

In [ ]:
# Layer 1
H = np.append(1, sigm(np.dot(W1,X))) # H = [1, h1, h2]
H

array([1.        , 0.92414182, 0.26894142])

In [ ]:
y = sigm(np.dot(W2,H))
y

0.5146489391238233

## Part (b)

In [ ]:
O = 1.0 # The target output
dE_dy = y - O # E = (y-O)^2 / 2.0

In [ ]:
dy_dW2 = dsigm(np.dot(W2,H)) * H
dy_dW2

array([0.24978541, 0.23083714, 0.06717764])

In [ ]:
dE_dW2 = dE_dy * dy_dW2
dE_dW2

array([-0.12123361, -0.11203705, -0.03260474])

In [ ]:
dh_dW1 = np.outer(dsigm(np.dot(W1,X)), X) # h=(h1, h2)
dh_dW1

array([[0.07010372, 0.        , 0.07010372],
       [0.19661193, 0.        , 0.19661193]])

In [ ]:
dy_dh = np.delete(dsigm(np.dot(W2,H)) * W2, 0)
dy_dh

array([0.24978541, 0.1248927 ])

In [ ]:
dE_dW1 = np.diag(dE_dy * dy_dh) @ dh_dW1
dE_dW1

array([[-0.00849893,  0.        , -0.00849893],
       [-0.01191799,  0.        , -0.01191799]])

In [ ]:
# One step update of Weights
alpha = 0.1
W2_new = W2 - alpha * dE_dW2
print("New W2 is ", W2_new)
W1_new = W1 - alpha * dE_dW1
print("New W1 is ", W1_new)

New W2 is  [-0.98787664  1.01120371  0.50326047]
New W1 is  [[ 1.50084989  2.5         1.00084989]
 [ 2.0011918  -1.5        -2.9988082 ]]


# Problem 2

## Tensor Class

In [7]:
# First import torch and check if GPU or TPU is avaliable. To accelerate by GPU or TPU, try Runtime -> Change runtime type
import torch as tr
print(tr.__version__)
if tr.cuda.is_available:
  print('cuda is avaliable')
else:
  print('cuda is no avaliable')

1.6.0+cu101
cuda is avaliable


In [ ]:
# Create a 2x3 tensor
x = tr.tensor([[1,2,3],[4,5,6]], device = 'cuda', dtype=tr.float32)

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')

In [ ]:
print(x,"\n")
print(x.type(),"\n")
print(x.size(),"\n")
print(x.shape,"\n")
print(x.dim(),"\n")

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0') 

torch.cuda.FloatTensor 

torch.Size([2, 3]) 

torch.Size([2, 3]) 

2 



In [ ]:
# Access elements
print('x[0,1]: ', x[0,1])
print('type(x[0,1]): ', type(x[0,1]))   # Still of torch type
print('type(x[0,1].item()): ', type(x[0,1].item())) #convert this to a Python scalar using the .item() method

x[0,1]:  tensor(2., device='cuda:0')
type(x[0,1]):  <class 'torch.Tensor'>
type(x[0,1].item()):  <class 'float'>



## Tensor constructors

PyTorch provides many convenience methods for constructing tensors; this avoids the need to use Python lists. For example:

- `torch.zeros`: Creates a tensor of all zeros
- `torch.ones`: Creates a tensor of all ones
- `torch.eye`: Creates a tensor of identity matrix
- `torch.rand`: Creates a tensor with uniform random numbers

You can find a full list of tensor creation operations in the [documentation](https://pytorch.org/docs/1.1.0/torch.html#creation-ops).


In [ ]:
# Create a tensor of all zeros
a = tr.zeros(2, 3, 4)
print('tensor of zeros:',a,'\n')

# Create a tensor of all ones
b = tr.ones(1, 2)
print('\ntensor of ones:',b,'\n')

# Create a 3x3 identity matrix
c = tr.eye(3)
print('\nidentity matrix:',c,'\n')

# Tensor of random values
d = tr.rand(3, 2, 2)
print('\nrandom tensor:',d,'\n')

tensor of zeros: tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]]) 


tensor of ones: tensor([[1., 1.]]) 


identity matrix: tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]) 


random tensor: tensor([[[0.8437, 0.9969],
         [0.2623, 0.1718]],

        [[0.2050, 0.5188],
         [0.6297, 0.4958]],

        [[0.3907, 0.6631],
         [0.3650, 0.4924]]]) 



## Pytorch Data Type
We can cast a tensor to another datatype using the `.to()` method; there are also convenience methods like `.float()` and `.long()` that cast to particular datatypes:

In [ ]:
x0 = tr.eye(3, dtype=tr.int64)
x1 = x0.float()  # Cast to 32-bit float
x2 = x0.double() # Cast to 64-bit float
x3 = x0.to(tr.float32) # Alternate way to cast to 32-bit float
x4 = x0.to(tr.float64) # Alternate way to cast to 64-bit float
print('x0:', x0.dtype)
print('x1:', x1.dtype)
print('x2:', x2.dtype)
print('x3:', x3.dtype)
print('x4:', x4.dtype)

x0: torch.int64
x1: torch.float32
x2: torch.float64
x3: torch.float32
x4: torch.float64




PyTorch provides several ways to create a tensor with the same datatype as another tensor:

- PyTorch provides tensor constructors such as `torch.new_zeros()` that create new tensors with the same shape and type as a given tensor
- Tensor objects have instance methods such as `.new_zeros()` that create tensors the same type but possibly different shapes
- The tensor instance method `.to()` can take a tensor as an argument, in which case it casts to the datatype of the argument.



In [ ]:
x0 = tr.eye(3, dtype=tr.float32)  # Shape (3, 3), dtype torch.float64
x1 = tr.zeros_like(x0)               # Shape (3, 3), dtype torch.float64
x2 = x0.new_zeros(4, 5)                 # Shape (4, 5), dtype torch.float64
x3 = tr.ones(6, 7).to(x0)            # Shape (6, 7), dtype torch.float64)
print('x0 shape is %r, dtype is %r' % (x0.shape, x0.dtype))
print('x1 shape is %r, dtype is %r' % (x1.shape, x1.dtype))
print('x2 shape is %r, dtype is %r' % (x2.shape, x2.dtype))
print('x3 shape is %r, dtype is %r' % (x3.shape, x3.dtype))

x0 shape is torch.Size([3, 3]), dtype is torch.float32
x1 shape is torch.Size([3, 3]), dtype is torch.float32
x2 shape is torch.Size([4, 5]), dtype is torch.float32
x3 shape is torch.Size([6, 7]), dtype is torch.float32



## Tensor indexing

Given index arrays idx0 and idx1 with N elements each, `a[idx0, idx1]` is equivalent to:
```
torch.tensor([
  a[idx0[0], idx1[0]],
  a[idx0[1], idx1[1]],
  ...,
  a[idx0[N - 1], idx1[N - 1]]
])
```
(A similar pattern extends to tensors with more than two dimensions)

Boolean tensor indexing lets you pick out arbitrary elements of a tensor according to a boolean mask. Frequently this type of indexing is used to select or modify the elements of a tensor that satisfy some condition.

In PyTorch, we use tensors of dtype `torch.uint8` to hold boolean masks; 0 means false and 1 means true.

In [ ]:
a = tr.tensor([[1,2], [3, 4], [5, 6]])
print('Original tensor:')
print(a)

print(a[[1,-1],[0,0]])
print(a[1:2,0:])

# Find the elements of a that are bigger than 3. The mask has the same shape as
# a, where each element of mask tells whether the corresponding element of a
# is greater than three.
mask = (a > 3)
print('\nMask tensor:')
print(mask)
# Check type of mask
print('\n', mask.type())

# We can use the mask to construct a rank-1 tensor containing the elements of a
# that are selected by the mask
print('\nSelecting elements with the mask:')
print(a[mask])

# We can also use boolean masks to modify tensors; for example this sets all
# elements <= 3 to zero:
a[a <= 3] = 0
print('\nAfter modifying with a mask:')
print(a)

Original tensor:
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([3, 5])
tensor([[3, 4]])

Mask tensor:
tensor([[False, False],
        [False,  True],
        [ True,  True]])

 torch.BoolTensor

Selecting elements with the mask:
tensor([4, 5, 6])

After modifying with a mask:
tensor([[0, 0],
        [0, 4],
        [5, 6]])



## Reshaping operations
View

PyTorch provides many ways to manipulate the shapes of tensors. The simplest example is `.view()`: This returns a new tensor with the same number of elements as its input, but with a different shape.

We can use `.view()` to flatten matrices into vectors, and to convert rank-1 vectors into rank-2 row or column matrices:


In [2]:
x0 = tr.tensor([[1, 2, 3, 4], [5, 6, 7, 8]])
print('Original tensor:')
print(x0)
print('shape:', x0.shape)

# Flatten x0 into a rank 1 vector of shape (8,)
x1 = x0.view(8)
print('\nFlattened tensor:')
print(x1)
print('shape:', x1.shape)

# Convert x1 to a rank 2 "row vector" of shape (1, 8)
x2 = x1.view(1, 8)
print('\nRow vector:')
print(x2)
print('shape:', x2.shape)

# Convert x1 to a rank 2 "column vector" of shape (8, 1)
x3 = x1.view(8, 1)
print('\nColumn vector:')
print(x3)
print('shape:', x3.shape)

# Convert x1 to a rank 3 tensor of shape (2, 2, 2):
x4 = x1.view(2, 2, 2)
print('\nRank 3 tensor:')
print(x4)
print('shape:', x4.shape)

Original tensor:
tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])
shape: torch.Size([2, 4])

Flattened tensor:
tensor([1, 2, 3, 4, 5, 6, 7, 8])
shape: torch.Size([8])

Row vector:
tensor([[1, 2, 3, 4, 5, 6, 7, 8]])
shape: torch.Size([1, 8])

Column vector:
tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8]])
shape: torch.Size([8, 1])

Rank 3 tensor:
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
shape: torch.Size([2, 2, 2])



### Swapping axes

Another common reshape operation you might want to perform is transposing a matrix. You might be surprised if you try to transpose a matrix with `.view()`: The `view()` function takes elements in row-major order, so you cannot transpose matrices with `.view()`.

In general, you should only use .view() to add new dimensions to a tensor, or to collapse adjacent dimensions of a tensor.

For other types of reshape operations, you usually need to use a function that can swap axes of a tensor. The simplest such function is `.t()`, specificially for transposing matrices. It is available both as a [function in the torch module](https://pytorch.org/docs/1.1.0/torch.html#torch.t), and as a [tensor instance method](https://pytorch.org/docs/1.1.0/tensors.html#torch.Tensor.t):


In [4]:
x = tr.tensor([[1, 2, 3], [4, 5, 6]])
print('Original matrix:')
print(x)
print('\nTransposing with view DOES NOT WORK!')
print(x.view(3, 2))
print('\nTransposed matrix:')
print(tr.t(x))
print(x.t())



Original matrix:
tensor([[1, 2, 3],
        [4, 5, 6]])

Transposing with view DOES NOT WORK!
tensor([[1, 2],
        [3, 4],
        [5, 6]])

Transposed matrix:
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])



### Contiguous tensors

Some combinations of reshaping operations will fail with cryptic errors. The exact reasons for this have to do with the way that tensors and views of tensors are implemented, and are beyond the scope of this assignment. However if you're curious, [this blog post by Edward Yang](http://blog.ezyang.com/2019/05/pytorch-internals/) gives a clear explanation of the problem.

What you need to know is that you can typically overcome these sorts of errors by either by calling `.contiguous()` before `.view()`, or by using `.reshape()` instead of `.view()`.


In [5]:
x0 = tr.randn(2, 3, 4)

try:
  # This sequence of reshape operations will crash
  x1 = x0.transpose(1, 2).view(8, 3)
except RuntimeError as e:
  print(type(e), e)
  
# We can solve the problem using either .contiguous() or .reshape()
x1 = x0.transpose(1, 2).contiguous().view(8, 3)
x2 = x0.transpose(1, 2).reshape(8, 3)
print('x1 shape: ', x1.shape)
print('x2 shape: ', x2.shape)

<class 'RuntimeError'> view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
x1 shape:  torch.Size([8, 3])
x2 shape:  torch.Size([8, 3])



## Tensor operations
### Elementwise operations

Basic mathematical functions operate elementwise on tensors, and are available as operator overloads, as functions in the torch module, and as instance methods on torch objects; all produce the same results:


In [9]:
x = tr.tensor([[1, 2, 3, 4]], dtype=tr.float32)
y = tr.tensor([[5, 6, 7, 8]], dtype=tr.float32)

# Elementwise sum; all give the same result
print('Elementwise sum:')
print(x + y)
print(tr.add(x, y))
print(x.add(y))

# Elementwise difference
print('\nElementwise difference:')
print(x - y)
print(tr.sub(x, y))
print(x.sub(y))

# Elementwise product
print('\nElementwise product:')
print(x * y)
print(tr.mul(x, y))
print(x.mul(y))

# Elementwise division
print('\nElementwise division')
print(x / y)
print(tr.div(x, y))
print(x.div(y))

# Elementwise power
print('\nElementwise power')
print(x ** y)
print(tr.pow(x, y))
print(x.pow(y))

Elementwise sum:
tensor([[ 6.,  8., 10., 12.]])
tensor([[ 6.,  8., 10., 12.]])
tensor([[ 6.,  8., 10., 12.]])

Elementwise difference:
tensor([[-4., -4., -4., -4.]])
tensor([[-4., -4., -4., -4.]])
tensor([[-4., -4., -4., -4.]])

Elementwise product:
tensor([[ 5., 12., 21., 32.]])
tensor([[ 5., 12., 21., 32.]])
tensor([[ 5., 12., 21., 32.]])

Elementwise division
tensor([[0.2000, 0.3333, 0.4286, 0.5000]])
tensor([[0.2000, 0.3333, 0.4286, 0.5000]])
tensor([[0.2000, 0.3333, 0.4286, 0.5000]])

Elementwise power
tensor([[1.0000e+00, 6.4000e+01, 2.1870e+03, 6.5536e+04]])
tensor([[1.0000e+00, 6.4000e+01, 2.1870e+03, 6.5536e+04]])
tensor([[1.0000e+00, 6.4000e+01, 2.1870e+03, 6.5536e+04]])


# Einstein Notation (`einsum`)
In mathematics, especially in applications of linear algebra to physics, the Einstein notation or Einstein summation convention is a notational convention that implies summation over a set of indexed terms in a formula, thus achieving notational brevity.

Einsum is implemented in numpy via `np.einsum`, in PyTorch via `torch.einsum`, and in TensorFlow via `tf.einsum`. All three einsum functions share the same signature `einsum(equation,operands)` where `equation` is a string representing the Einstein summation and operands is a sequence of tensors. Check [tech blog by Tim Rocktäschel](https://rockt.github.io/2018/04/30/einsum).

For instance, our first example $c_j=\sum_i\sum_k A_{ik}B_{kj}$ can be written as the equation string `"ik,kj -> j"`. Note that the naming of the indices $(i, j, k)$ is arbitrary but it needs to be used consistently. 

 What's great about having einsum not only in numpy but also in PyTorch and TensorFlow is that it can be used in arbitrary computation graphs for neural network architectures and that we can backpropagate through it. A typical call to einsum has the following form
$$result=einsum("◻◻,◻◻◻,◻◻->◻◻",arg1,arg2,arg3)$$
where $◻$ is a placeholder for a character identifying a tensor dimension. From this equation string we can infer that `arg1` and `arg3` are matrices, `arg2` is an order-3 tensor, and that the result of this einsum operation is a matrix.

## 1 Matrix Transpose
$$B_{ji}=A_{ij}$$

In [15]:
a = tr.arange(6).reshape(2,3)
print(a)
b = tr.einsum('ij->ji', a)
print(b)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


## 2. Sum
### 2.1 Total Summation
$$\sum_i\sum_j A_{ij}$$

In [18]:
tr.einsum('ij->',a)

tensor(15)

### 2.2 Column Sum
$$b_j = \sum_i A_{ij}$$

In [20]:
tr.einsum('ij->j',a)

tensor([3, 5, 7])

### 2.3 Row Sum
$$a_i = \sum_j A_{ij}$$

In [21]:
tr.einsum('ij->i',a)

tensor([ 3, 12])

### 2.4 Matrix times Vector
$c_i = \sum_j A_{ij}b_j:=A_{ij}b_j$

In [26]:
A = tr.arange(6).reshape(2,3)
b = tr.arange(3)
c = tr.einsum('ij,j->i',A,b)
print("A:",A)
print("b:",b)
print("c:",c)

A: tensor([[0, 1, 2],
        [3, 4, 5]])
b: tensor([0, 1, 2])
c: tensor([ 5, 14])


### 2.5 Matrix times Matrix
$$C_{ij} = \sum_k A_{ik}B_{kj}:=A_{ik}B_{kj}$$

In [27]:
A = tr.arange(6).reshape(2,3)
B = tr.arange(12).reshape(3,4)
C = tr.einsum('ik,kj->ij',A,B)
print("A:",A)
print("B:",B)
print("C:",C)

A: tensor([[0, 1, 2],
        [3, 4, 5]])
B: tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
C: tensor([[20, 23, 26, 29],
        [56, 68, 80, 92]])


### 2.6 Dot Product
$$c = \sum_i\sum_j A_{ij}B_{ij}$$

In [29]:
A = tr.arange(6).reshape(2,3)
B = tr.arange(6).reshape(2,3)
c = tr.einsum('ik,ik->',A,B)
print("A:",A)
print("B:",B)
print("c:",c)

A: tensor([[0, 1, 2],
        [3, 4, 5]])
B: tensor([[0, 1, 2],
        [3, 4, 5]])
c: tensor(55)


### 2.7 Hadamard Product
$$C_{ij} = A_{ij}B_{ij}$$

In [30]:
A = tr.arange(6).reshape(2,3)
B = tr.arange(6).reshape(2,3)
C = tr.einsum('ij,ij->ij',A,B)
print("A:",A)
print("B:",B)
print("C:",C)

A: tensor([[0, 1, 2],
        [3, 4, 5]])
B: tensor([[0, 1, 2],
        [3, 4, 5]])
C: tensor([[ 0,  1,  4],
        [ 9, 16, 25]])


### 2.8 Outer Product
$$C_{ij} = a_i b_j $$
$$\text{i.e., }C = ab^{\intercal}$$

In [32]:
a = tr.arange(3)
b = tr.arange(3,7)  # -- a vector of length 4 containing [3, 4, 5, 6]
print(a)
print(b)
tr.einsum('i,j->ij', [a, b])

tensor([0, 1, 2])
tensor([3, 4, 5, 6])


tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])

### 2.9 Batch Matrix Multiplication
$$C_{ijl} = \sum_{k} A_{ijk}B_{ikl}$$

In [33]:
a = tr.randn(3,2,5)
b = tr.randn(3,5,3)
print(a)
print(b)
tr.einsum('ijk,ikl->ijl', [a, b])

tensor([[[ 1.6552,  0.1795, -2.3108,  0.3509, -2.4135],
         [-0.8157, -1.3631,  0.7683, -0.8668,  0.1366]],

        [[ 0.9263, -0.2823, -1.2426,  1.4459,  0.7716],
         [-0.4063, -0.6770, -0.8384, -0.1709,  0.3847]],

        [[-0.8797,  0.8190,  0.9115, -0.0935, -1.7566],
         [ 1.7730, -0.0858, -1.3908, -0.3554,  1.2814]]])
tensor([[[ 0.0729,  0.6197,  0.8966],
         [-0.8856,  0.0655, -1.2936],
         [-1.5492,  0.4757, -0.9978],
         [-1.3782, -1.4919, -1.2327],
         [-0.1998, -0.0917,  0.6421]],

        [[ 1.5846,  0.4726, -0.0457],
         [ 0.9825,  1.6021, -0.4993],
         [-1.3244,  0.8215, -0.1822],
         [ 2.6278,  1.9126, -1.9630],
         [-0.0788,  0.6109, -0.0908]],

        [[ 0.0265, -0.2697,  1.6345],
         [-0.3073,  0.7065, -1.7149],
         [ 1.7637, -0.0542,  0.1717],
         [ 2.2155,  0.0635,  0.1447],
         [ 1.2697, -0.0547,  2.6335]]])


tensor([[[ 3.5401, -0.3641,  1.5754],
         [ 1.1248,  1.0514,  1.4215]],

        [[ 6.5748,  2.2015, -2.5833],
         [-0.6780, -2.0572,  0.8098]],

        [[-1.1049,  0.8567, -7.3254],
         [-1.5401, -0.5563,  6.1294]]])

### 2.10 Tensor Contraction
For example, if we have two tensors, an order-$n$tensor ${\cal A}\in\mathbb{R}^{I_1\times\dots\times I_n}$ and an order-$m$ tensor ${\cal B}\in\mathbb{R}^{J_1\times\dots\times J_m}$, where $n=4,m=5$ and assume that $I_2=J_3$ and $I_3=J_5$. We can multiply the two tensors in these two dimensions (2,3 of ${\cal A}$ and 3,5 of ${\cal B}$) resulting in anew tensor ${\cal C}\in\mathbb{R}^{I_1\times I_4 \times J_1\times J_2 \times J_4}$:
# $$C_{pstuv} = \sum_q\sum_r A_{pqrs}B_{tuqvr}$$

In [35]:
A = tr.randn(2,3,5,7)
B = tr.randn(11,13,3,17,5)
tr.einsum('pqrs,tuqvr->pstuv', [A, B]).shape

torch.Size([2, 7, 11, 13, 17])

### 2.11 Bilinear Transformation
$$D_{ij} = \sum_k\sum_l A_{ik}B_{jkl}C_{il}:=A_{ik}B_{jkl}C_{il}$$


In [37]:
a = tr.randn(3,4)
b = tr.randn(5,4,7)
c = tr.randn(3,7)
tr.einsum('ik,jkl,il->ij', [a, b, c])

tensor([[ 3.7615,  0.7483, -1.1445, -1.7901,  3.2181],
        [ 6.5693,  2.6666, -0.2932, -1.6851,  0.7707],
        [ 0.4451,  1.1593, -1.3036,  4.0214,  3.3757]])

### 2.12*
New Update: Diagnal of a matrix
$$A_{ii}$$

In [38]:
a = tr.randn(3,3)
print(a)
tr.einsum('ii->i',a)

tensor([[-0.5258, -0.2254,  1.0018],
        [-0.8506,  1.2909,  0.4217],
        [ 0.4302,  1.1059,  0.6009]])


tensor([-0.5258,  1.2909,  0.6009])